In [12]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

from xgboost.sklearn import XGBClassifier

In [6]:
emotions_df = pd.read_csv('data/emotions.csv')

In [7]:
emotions_df

,# mean_0_a,mean_1_a,mean_2_a,mean_3_a,mean_4_a,mean_d_0_a,mean_d_1_a,mean_d_2_a,mean_d_3_a,mean_d_4_a,...,fft_741_b,fft_742_b,fft_743_b,fft_744_b,fft_745_b,fft_746_b,fft_747_b,fft_748_b,fft_749_b,label
0,4.620,30.3,-356.0,15.60,26.3,1.070,0.411,-15.700,2.060,3.15,...,23.50,20.300,20.300,23.50,-215.0,280.00,-162.00,-162.00,280.00,NEGATIVE
1,28.800,33.1,32.0,25.80,22.8,6.550,1.680,2.880,3.830,-4.82,...,-23.30,-21.800,-21.800,-23.30,182.0,2.57,-31.60,-31.60,2.57,NEUTRAL
2,8.900,29.4,-416.0,16.70,23.7,79.900,3.360,90.200,89.900,2.03,...,462.00,-233.000,-233.000,462.00,-267.0,281.00,-148.00,-148.00,281.00,POSITIVE
3,14.900,31.6,-143.0,19.80,24.3,-0.584,-0.284,8.820,2.300,-1.97,...,299.00,-243.000,-243.000,299.00,132.0,-12.40,9.53,9.53,-12.40,POSITIVE
4,28.300,31.3,45.2,27.30,24.5,34.800,-5.790,3.060,41.400,5.52,...,12.00,38.100,38.100,12.00,119.0,-17.60,23.90,23.90,-17.60,NEUTRAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2127,32.400,32.2,32.2,30.80,23.4,1.640,-2.030,0.647,-0.121,-1.10,...,-21.70,0.218,0.218,-21.70,95.2,-19.90,47.20,47.20,-19.90,NEUTRAL
2128,16.300,31.3,-284.0,14.30,23.9,4.200,1.090,4.460,4.720,6.63,...,594.00,-324.000,-324.000,594.00,-35.5,142.00,-59.80,-59.80,142.00,POSITIVE
2129,-0.547,28.3,-259.0,15.80,26.7,9.080,6.900,12.700,2.030,4.64,...,370.00,-160.000,-160.000,370.00,408.0,-169.00,-10.50,-10.50,-169.00,NEGATIVE
2130,16.800,19.9,-288.0,8.34,26.0,2.460,1.580,-16.000,1.690,4.74,...,124.00,-27.600,-27.600,124.00,-656.0,552.00,-271.00,-271.00,552.00,NEGATIVE


In [8]:
emotions_df.shape

(2132, 2549)

In [57]:
# check if there are any missing values
is_missing = False
for x in emotions_df.isnull().sum():
    if x > 0:
        is_missing = True
        break
if is_missing:
    print('There are missing values in the dataset')
else:
    print('No missing values in the dataset')

No missing values in the dataset


In [59]:
# check number of values in each label
emotions_df['label'].value_counts()

label
NEUTRAL     716
NEGATIVE    708
POSITIVE    708
Name: count, dtype: int64

In [50]:
X = emotions_df.drop(['label'], axis=1).to_numpy()
y = emotions_df[['label']].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.15)

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

encoder = OneHotEncoder()
y_train_enc = np.array(encoder.fit_transform(y_train).todense())
y_test_enc = np.array(encoder.transform(y_test).todense())

In [76]:
# Logistic Regression
logreg = LogisticRegression()
logreg.fit(X_train_std, y_train)
y_pred_logreg = logreg.predict(X_test_std)
acc_logreg = accuracy_score(y_test, y_pred_logreg)

# K-Nearest Neighbors
knn = KNeighborsClassifier()
knn.fit(X_train_std, y_train)
y_pred_knn = knn.predict(X_test_std)
acc_knn = accuracy_score(y_test, y_pred_knn)

# Support Vector Machine
svm = SVC()
svm.fit(X_train_std, y_train)
y_pred_svm = svm.predict(X_test_std)
acc_svm = accuracy_score(y_test, y_pred_svm)

# Decision Tree
dtree = DecisionTreeClassifier()
dtree.fit(X_train_std, y_train)
y_pred_dtree = dtree.predict(X_test_std)
acc_dtree = accuracy_score(y_test, y_pred_dtree)

# XGBoost
xgb = XGBClassifier()
xgb.fit(X_train_std, y_train_enc)
y_pred_xgb = xgb.predict(X_test_std)
acc_xgb = accuracy_score(y_test_enc, y_pred_xgb)


C:\Users\hubra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\hubra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\hubra\AppData\Local

In [77]:
# Accuracy for each classifier
print("Logistic Regression Accuracy: {:.2f}%".format(acc_logreg * 100))
print("K-Nearest Neighbors Accuracy: {:.2f}%".format(acc_knn * 100))
print("Support Vector Machine Accuracy: {:.2f}%".format(acc_svm * 100))
print("Decision Tree Accuracy: {:.2f}%".format(acc_dtree * 100))
print("XGBoost Accuracy: {:.2f}%".format(acc_xgb * 100))

Logistic Regression Accuracy: 98.12%
K-Nearest Neighbors Accuracy: 95.62%
Support Vector Machine Accuracy: 96.88%
Decision Tree Accuracy: 95.00%
XGBoost Accuracy: 98.12%
